In [154]:
import requests
from bs4 import BeautifulSoup
import re
import warnings
import json
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

# GET LINKs

In [58]:
path = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
r = requests.get(f'{path}')
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('tbody', class_ = 'lister-list')
movies = s.find_all('td',class_ = 'titleColumn')
links = []
base = 'https://www.imdb.com'
for movie in movies:
    link = movie.a['href']
    links.append(f'{link}')

save links

In [36]:
links = np.array(links)
np.save('top_250_Linkes',links)

# GET DATA

In [172]:
headers = {'User-Agent': 'Safari/605.1.15'}
xrror = []

movies = pd.DataFrame(columns = ['id','title','year','runtime','parental_guide','gross_us_canda'])
genre_movie =  pd.DataFrame(columns = ['movie_id','genre'])
cast = ['movie_id','person_id']
crew = ['movie_id','person_id','role']

TODO : add cast and crew

In [188]:
for link in links[32:40]:
    r = requests.get(f'{base}{link}', headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    try:
        movie_id = link[9:-1]
        title = soup.find('span', class_ = 'sc-afe43def-1 fDTGTb').text
        info = soup.find_all('li',role ='presentation',class_="ipc-inline-list__item")
        year = int(info[3].text)
        parental_guide = info[4].text
        runtime = info[5].text
        gross_us_canda = soup.find_all('span', class_ = 'ipc-metadata-list-item__list-content-item')[3].text
        movies.loc[len(movies.index)] = [movie_id,title,year,runtime,parental_guide,gross_us_canda]
        genres = soup.find_all('a', class_ = 'ipc-chip ipc-chip--on-baseAlt')
        for g in genres:
            genre_movie.loc[len(genre_movie.index)] = [movie_id,g.text] 
#         casts = 
#         for cast in casts:
#             person_id = 
#             cast.loc[len(cast.index)] = [movie_id,person_id] 
#             full_name = 
#             person.loc[len(person.index)] = [person_id,full_name]
#             # must be cleaned (remove duplicates)
#         each_crew = 
#         for c in each_crew:
#             person_id = 
# #             check_type
#             role = 
#             crew.loc[len(crew.index)] = [movie_id,person_id,role] 
#             full_name = 
#             person.loc[len(person.index)] = [person_id,full_name]
#             # must be cleaned (remove duplicates)
    except Exception as e: 
        print(e)       
        xrror.append(link)


save links[0:40] data (without cast and crew)

In [ ]:
genres.to_csv('genres(re.zip',index=False)
movies.to_csv(index=False)